In [1]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly

from datetime import timedelta
import datetime as dt
today = dt.datetime.today().date()
yesterday = today - timedelta(days=1)
import calendar

from Position_Report_Calcs import Position_Reporting

In [2]:
plotly.offline.init_notebook_mode(connected=True)

In [3]:
## Read in the position data
mkts = ['ACCU','LGC','NZU','EUA','UKA','CCA','VCM']
products = ['EUA Dec24','UKA Dec24','ACCU spot','LGC Spot','NZU spot','CCA V24 Dec24']
products_map = ['EUA','UKA','ACCU','LGC','NZU','CCA']

positions = dict()
for m in mkts:
    positions[m] = pd.read_excel('Positions.xlsx', sheet_name=m)
    positions[m]['Expiry'] = pd.to_datetime(positions[m].Expiry).dt.date

premiums = pd.read_excel('Positions.xlsx', sheet_name='Index')
FUM = premiums[premiums.Spread=='FUM'].Price.reset_index()['Price'][0]

returns = pd.read_excel('Positions.xlsx', sheet_name='Returns')   # this is FUM returns
returns[list(returns)[1]] = round(returns[list(returns)[1]],2)
for c in list(returns)[2:]:
    returns[c] = round(returns[c]*100,2)
returns[list(returns)[1]] = ["${:,.2f}".format(i) for i in returns[list(returns)[1]]] # add a $ sign to the pnl column
for c in list(returns)[2:]:    # add a % sign to the returns columns 
    returns[c] = ["{:.2%}".format(i/100) for i in returns[c]]
#returns = returns.set_index('Market')


prices = pd.read_excel('Positions.xlsx', sheet_name='Prices')
#prices = prices[(prices['Date'].dt.year==today.year) & (prices['Date'].dt.month==today.month)]
#prices['Date'] = [i.date() for i in prices.Date]
#prices = prices[prices.Date <= yesterday].reset_index(drop=True)
prices = prices.dropna()

In [4]:
portfolio_allocation = pd.read_excel('Positions.xlsx', sheet_name='Allocation')
portfolio_allocation = portfolio_allocation.set_index('Market')

portfolio_allocation['Required Delta'] = round(portfolio_allocation['Required Delta'])
portfolio_allocation['Current Delta'] = round(portfolio_allocation['Current Delta'])

portfolio_allocation[list(portfolio_allocation)[0]] = ["${:,.2f}".format(i) for i in portfolio_allocation[list(portfolio_allocation)[0]]] # add a $ sign to the 1% move column
portfolio_allocation[list(portfolio_allocation)[1]] = ["${:,.2f}".format(i) for i in portfolio_allocation[list(portfolio_allocation)[1]]] # add a $ sign to the AUD required return

portfolio_allocation[list(portfolio_allocation)[-2]] = ["{:,.2%}".format(i) for i in portfolio_allocation[list(portfolio_allocation)[-2]]] # add a % sign to the actual weight column
portfolio_allocation[list(portfolio_allocation)[-1]] = ["{:,.2%}".format(i) for i in portfolio_allocation[list(portfolio_allocation)[-1]]] # add a % sign to the over / under weight column

portfolio_allocation = portfolio_allocation.reset_index()

In [5]:
## Define the dates we want to use for reporting
current_date = dt.datetime.today().date()
current_year = str(dt.datetime.today().date().year)

_, last_day = calendar.monthrange(current_date.year, current_date.month)
end_of_month = dt.date(current_date.year, current_date.month, last_day)

one_month = current_date + dt.timedelta(days=30)

three_months = current_date + dt.timedelta(days=3*30)

six_months = current_date + dt.timedelta(days=6*30)

eoy = '10-12-{}'.format(current_year)
end_of_year = dt.datetime.strptime(eoy, '%d-%m-%Y').date()

end_of_feb = dt.datetime.strptime('29-02-2024', '%d-%m-%Y').date()

one_year = current_date + dt.timedelta(days=365)

dates_names = ['today','EoM','3 months','6 months','EoY','OneYear','End of Feb']
dates_values = [current_date, end_of_month, three_months, six_months, end_of_year, one_year, end_of_feb]

In [6]:
report = Position_Reporting(positions, 'EUA', current_date)   # This is a general instance of the class to pull global variables from
fx_rates = report.fx_all

In [7]:
dates = dict()
for i in list(range(0,len(dates_names))):
    dates[dates_names[i]] = dates_values[i]

In [8]:
euas = dict()
accus = dict()
lgcs = dict()
nzus = dict()
ukas = dict()
ccas = dict()

for d in dates_values:
    euas[d] = Position_Reporting(positions, 'EUA', d).combine_frame()
    accus[d] = Position_Reporting(positions, 'ACCU', d).combine_frame()
    lgcs[d] = Position_Reporting(positions, 'LGC', d).combine_frame()
    nzus[d] = Position_Reporting(positions, 'NZU', d).combine_frame()
    ukas[d] = Position_Reporting(positions, 'UKA', d).combine_frame()
    ccas[d] = Position_Reporting(positions, 'CCA', d).combine_frame()

In [9]:
# Make a bar chart of the current allocation for each mkt at the current price
eua_allocation = Position_Reporting(positions, 'EUA', current_date).current_values()
nzu_allocation = Position_Reporting(positions, 'NZU', current_date).current_values()
accu_allocation = Position_Reporting(positions, 'ACCU', current_date).current_values()
lgc_allocation = Position_Reporting(positions, 'LGC', current_date).current_values()
uka_allocation = Position_Reporting(positions, 'UKA', current_date).current_values()
cca_allocation = Position_Reporting(positions, 'CCA', current_date).current_values()
vcm_allocation = Position_Reporting(positions, 'VCM', current_date).current_values()

allocation_frame = pd.DataFrame()
#allocation_frame['Type'] = ['Spot','Fwd','Option']
allocation_frame['ACCU'] = accu_allocation
allocation_frame['LGC'] = lgc_allocation
allocation_frame['NZU'] = nzu_allocation
allocation_frame['EUA'] = eua_allocation
allocation_frame['UKA'] = uka_allocation
allocation_frame['CCA'] = cca_allocation
allocation_frame['VCM'] = vcm_allocation

allocation_frame = allocation_frame.append(allocation_frame.sum(), ignore_index=True)
allocation_frame['Type'] = ['Spot','Fwd','Option','Total']
allocation_frame = allocation_frame.set_index('Type')

allocation_frame = allocation_frame.T

mkt_allocation = list(allocation_frame.Total)
mkt_allocation = [round(i/FUM,2) for i in mkt_allocation]

#fig = go.Figure()
#for i in list(allocation_frame)[:-1]:
#    fig.add_trace(go.Bar(x=allocation_frame.index, y=allocation_frame[i], name=i, base=0))
#fig.add_trace(go.Scatter(x=allocation_frame.index, y=allocation_frame.Total, name='Total $', line=dict(color='red')))
#
#fig.add_trace(go.Scatter(x=allocation_frame.index, y=mkt_allocation, name='% FUM', line=dict(color='green'), yaxis='y2'))
#
## Add the second y-axis
#fig.update_layout(
#    yaxis2=dict(
#        title='Y-axis 2 Title',
#        overlaying='y',
#        side='right'
#    )
#)
#
#fig.update_layout(barmode='stack', title='Net Allocation (AUD)')
#fig.show()

In [10]:
current_prices = report.current_prices.reset_index(drop=True)
rounded_prices = current_prices.copy()
rounded_prices['Price'] = rounded_prices.Price.round()

price_dict = {}
for mkt in list(rounded_prices.Spread):
    price_dict[mkt] = rounded_prices[rounded_prices.Spread==mkt].Price.reset_index().Price[0]

In [11]:
# dv = delta value
accu_dv = accus[current_date][1]
accu_dv = accu_dv[accu_dv.Price==price_dict['ACCU']]
accu_dv['FX'] = 1

lgc_dv = lgcs[current_date][1]
lgc_dv = lgc_dv[lgc_dv.Price==price_dict['LGC']]
lgc_dv['FX'] = 1

nzu_dv = nzus[current_date][1]
nzu_dv = nzu_dv[nzu_dv.Price==price_dict['NZU']]
nzu_dv['FX'] = fx_rates[fx_rates['Spread']=='NZDAUD'].reset_index()['Price'][0]

eua_dv = euas[current_date][1]
eua_dv = eua_dv[eua_dv.Price==price_dict['EUA']]
eua_dv['FX'] = fx_rates[fx_rates['Spread']=='EURAUD'].reset_index()['Price'][0]

uka_dv = ukas[current_date][1]
uka_dv = uka_dv[uka_dv.Price==price_dict['UKA']]
uka_dv['FX'] = fx_rates[fx_rates['Spread']=='GBPAUD'].reset_index()['Price'][0]

cca_dv = ccas[current_date][1]
cca_dv = cca_dv[cca_dv.Price==price_dict['CCA']]
cca_dv['FX'] = fx_rates[fx_rates['Spread']=='USDAUD'].reset_index()['Price'][0]

In [12]:
delta_values = pd.concat([accu_dv, lgc_dv, nzu_dv, eua_dv, uka_dv, cca_dv]).reset_index(drop=True)
for c in list(delta_values)[1:-1]:
    delta_values[c] = delta_values['Price'] * delta_values[c]   # convert the delta into montery value
    delta_values[c] *= delta_values['FX']
    delta_values[c] = delta_values[c].round()
    

delta_values['Mkt'] = ['ACCU','LGC','NZU','EUA','UKA','CCA']


#~~~~~
# Melt the delta values so that it is usefule for charts
b = delta_values.copy()
b = b.drop(columns='FX')
#b.head()
m = pd.melt(b, id_vars=['Mkt'], value_vars=list(b)[1:-2], var_name='Type')
#m.head()
b['FUM'] = round((b['Total_Delta']/FUM)*100,1) # this has to go after the transformation above

delta_values['FUM'] = round(delta_values['Total_Delta']/FUM * 100, 1) 

In [41]:
fig = make_subplots(
    rows=1, cols=3,
    #shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"},
           {"type": "table"},
           {"type": "scatter"}]]
)

fig.add_trace(
    go.Table(
        header=dict(
            values=list(returns),
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[returns[k].tolist() for k in returns.columns],
            align = "left")
    ),
    row=1, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=list(portfolio_allocation),
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[portfolio_allocation[k].tolist() for k in portfolio_allocation.columns],
            align = "left")
    ),
    row=1, col=2
)

barcols=['Spot','Fwds','Options']
for i, col in enumerate(barcols):
    fig.add_trace(
        go.Bar(
            x=delta_values['Mkt'],
            y=delta_values[col],
            name=col
        ),
        row=1, col=3
)
    
fig.add_trace(
    go.Scatter(
        x=delta_values['Mkt'],
        y=delta_values['Total_Delta'], name='Total Delta'
    ),
    row=1, col=3
)


fig.update_layout(
    height=400,
    width=1950,
    showlegend=True,
    title_text="Monthly Return and Current Allocation",
    barmode='stack'
)

fig.show() 

# TO DO
### Need to look at what this portfolio would return against what an equally weighted portfolio would return.
### Use historical return data

In [14]:
## CREATE PNL SCENARIOS FOR INPUT PRICES ##
# low / high prices for testing... need to be manually adjusted in notebook for time being
accu = [28,41]
lgc = [35,46]
nzu = [60,81]
eua = [35,71]
uka = [30,41]
cca = [39,46]

low_price = []
high_price = []

prices = [accu,lgc,nzu,eua,uka,cca]
mkts = ['ACCU','LGC','NZU','EUA','UKA','CCA']

price_frame_today = pd.DataFrame()
price_frame_OneMonth = pd.DataFrame()

pnl_graph_frame_today = pd.DataFrame()
pnl_graph_frame_OneMonth = pd.DataFrame()

mkt_today = []
high_today = []
low_today = []

mkt_OneMonth = []
high_OneMonth = []
low_OneMonth = []
for i in range(0,len(prices)):
    low_price.append(prices[i][0])
    high_price.append(prices[i][1])
    
    mkt_today_report = Position_Reporting(positions, mkts[i], current_date)
    mkt_one_month_report = Position_Reporting(positions, mkts[i], one_month)
    pnls_today = mkt_today_report.std_moves(1,'daily', prices[i])
    pnls_OneMonth = mkt_one_month_report.std_moves(1,'daily', prices[i])    
    
    mkt_today.append(mkts[i])
    low_today.append(round(pnls_today['Total'][0],2))
    high_today.append(round(pnls_today['Total'][1],2))
    
    mkt_OneMonth.append(mkts[i])
    low_OneMonth.append(round(pnls_OneMonth['Total'][0],2))
    high_OneMonth.append(round(pnls_OneMonth['Total'][1],2))    
    
price_frame_today['Mkt'] = mkts
price_frame_today['Low Price'] = low_today
price_frame_today['High Price'] = high_today

price_frame_OneMonth['Mkt'] = mkts
price_frame_OneMonth['Low Price'] = low_OneMonth
price_frame_OneMonth['High Price'] = high_OneMonth

prices_used = pd.DataFrame()
prices_used['Market'] = mkts
prices_used['Low Price'] = low_price
prices_used['High Price'] = high_price

In [15]:
widths = [10] * len(prices_used.columns)
fig = ff.create_table(prices_used, height_constant=12)
fig.layout.width=300
fig.show()

In [16]:
delta_graph = px.bar(m, x='Mkt', y='value',color='Type')
delta_graph.add_trace(go.Scatter(x=b.Mkt, y=b['Total_Delta'],name='Total Allocation'))#, name='Total Allocation', base=0))
delta_graph.update_layout(title='Delta Value (AUD)')

delta_graph.add_trace(go.Scatter(x=b.Mkt, y=b.FUM, name='% FUM', line=dict(color='green'), yaxis='y2'))
# Add the second y-axis
delta_graph.update_layout(
    yaxis2=dict(
        title='Y-axis 2 Title',
        overlaying='y',
        side='right'
    )
)
delta_graph.show()       

In [17]:
pnl_graph = go.Figure()
for i in list(price_frame_today)[1:]:
    pnl_graph.add_trace(go.Bar(x=price_frame_today.Mkt, y=price_frame_today[i], name=i, base=0))
pnl_graph.update_layout(barmode='stack', title='PnL Scenarios (Today)')
pnl_graph.show()

In [18]:
#fig = go.Figure()
#for i in list(price_frame_OneMonth)[1:]:
#    fig.add_trace(go.Bar(x=price_frame_OneMonth.Mkt, y=price_frame_OneMonth[i], name=i, base=0))
#fig.update_layout(barmode='stack', title='PnL Scenarios (One Month)')
#fig.show()

# Market Risk Allocation

In [19]:
# Have Pnl on left axis, % fum on right
price_returns = [-0.15, -0.1, -0.05, 0, .05, .1, .15]

def create_returns(horizon):
    price_pnls = pd.DataFrame()
    price_ranges = pd.DataFrame()
    price_ranges['% Move'] = [int(i*100) for i in price_returns]
    for m in mkts:    #[:-1]:
        mkt_current = Position_Reporting(positions, m, horizon)
        spot = mkt_current.spot_price
        mkt_prices = [(1+i)*spot for i in price_returns]
        price_pnls[m] = round(mkt_current.price_moves(mkt_prices)['Total'],2)   
        price_ranges[m] = mkt_prices

    price_ranges = round(price_ranges,2)
    return price_ranges, price_returns, price_pnls

In [20]:
price_ranges, price_returns, price_pnls = create_returns(current_date)

fig = ff.create_table(price_ranges,height_constant=50)

for i in mkts:
    #fig.add_trace(go.Scatter(x=fig_daily.Sigma, y=fig_daily[i], name=i, xaxis='x2', yaxis='y2'))
    fig.add_trace(go.Scatter(x = price_returns, y=price_pnls[i], name=i, xaxis='x2', yaxis='y2'))
    fig.update_xaxes(tickvals=[-0.15,-0.10,-0.5,0,0.5,0.10,0.15])
    
fig.update_layout(
    title_text = 'Theoretical Prices - Today',
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.],'title':''},
    yaxis2 = {'anchor': 'x2', 'title': 'PnL (AUD)'}
)

fig.show()

In [21]:
allocation = pd.DataFrame()   ## T%HIS IS SUPPOSED TO BE LIKE THE TABLE I HAVE PUT IN MONTHLY MTM AND IT SHOULD PROBABLY BE UP TOWARDS THE TOP OF THE NOTEBOOK SOMEWHERE
allocation['Market'] = mkts

In [22]:
1/len(mkts)

0.16666666666666666

In [23]:
fum_frame = pd.DataFrame()

fum_frame['% Move'] = price_returns

for i in mkts:
    pnls = price_pnls[i]   # This is the $ return at each % move
    pnls = [(i/FUM)*100 for i in pnls]
    pnls = [str(round(i,2)) for i in pnls]
    pnls = [i+'%' for i in pnls]
    fum_frame[i] = pnls

In [24]:
price_pnls[i]

0   -567997.79
1   -396281.22
2   -208191.00
3     -5319.55
4    209234.06
5    431384.54
6    656673.88
Name: CCA, dtype: float64

In [25]:
fum_frame    ## UP TO HERE

,% Move,ACCU,LGC,NZU,EUA,UKA,CCA
0,-0.15,1.61%,-0.87%,-4.83%,-0.17%,-1.36%,-3.36%
1,-0.10,1.01%,-0.58%,-3.18%,-0.14%,-0.87%,-2.35%
2,-0.05,0.47%,-0.29%,-1.61%,-0.08%,-0.37%,-1.23%
3,0.00,0.0%,0.0%,0.0%,0.04%,0.15%,-0.03%
4,0.05,-0.42%,0.29%,1.75%,0.2%,0.68%,1.24%
5,0.10,-0.79%,0.58%,3.7%,0.41%,1.21%,2.56%
6,0.15,-1.12%,0.87%,5.83%,0.66%,1.76%,3.89%


In [26]:
price_ranges, price_returns, price_pnls = create_returns(end_of_month)

fig = ff.create_table(price_ranges,height_constant=50)

for i in mkts:
    #fig.add_trace(go.Scatter(x=fig_daily.Sigma, y=fig_daily[i], name=i, xaxis='x2', yaxis='y2'))
    fig.add_trace(go.Scatter(x = price_returns, y=price_pnls[i], name=i, xaxis='x2', yaxis='y2'))
    fig.update_xaxes(tickvals=[-0.15,-0.10,-0.5,0,0.5,0.10,0.15])
    
fig.update_layout(
    title_text = 'Theoretical Prices - End of Month',
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.],'title':''},
    yaxis2 = {'anchor': 'x2', 'title': 'PnL (AUD)'}
)

fig.show()

In [27]:
price_ranges, price_returns, price_pnls = create_returns(three_months)

fig = ff.create_table(price_ranges,height_constant=50)

for i in mkts:
    #fig.add_trace(go.Scatter(x=fig_daily.Sigma, y=fig_daily[i], name=i, xaxis='x2', yaxis='y2'))
    fig.add_trace(go.Scatter(x = price_returns, y=price_pnls[i], name=i, xaxis='x2', yaxis='y2'))
    fig.update_xaxes(tickvals=[-0.15,-0.10,-0.5,0,0.5,0.10,0.15])
    
fig.update_layout(
    title_text = 'Theoretical Prices - Three Months',
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.],'title':''},
    yaxis2 = {'anchor': 'x2', 'title': 'PnL (AUD)'}
)

fig.show()

In [28]:
price_ranges, price_returns, price_pnls = create_returns(six_months)

fig = ff.create_table(price_ranges,height_constant=50)

for i in mkts:
    #fig.add_trace(go.Scatter(x=fig_daily.Sigma, y=fig_daily[i], name=i, xaxis='x2', yaxis='y2'))
    fig.add_trace(go.Scatter(x = price_returns, y=price_pnls[i], name=i, xaxis='x2', yaxis='y2'))
    fig.update_xaxes(tickvals=[-0.15,-0.10,-0.5,0,0.5,0.10,0.15])
    
fig.update_layout(
    title_text = 'Theoretical Prices - Six Months',
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.],'title':''},
    yaxis2 = {'anchor': 'x2', 'title': 'PnL (AUD)'}
)

fig.show()

### TO DO - this should use percentiles not std dev because the interpretation of st dev assumes normal dist

In [29]:
def sigma_moves(mkt, calc_numbers=''):
    mkt_current = Position_Reporting(positions, mkt, current_date)
    
    sub_1s_daily = mkt_current.std_moves(1, 'daily', calc_numbers)
    sub_15s_daily = mkt_current.std_moves(1.5, 'daily', calc_numbers)
    sub_2s_daily = mkt_current.std_moves(2, 'daily', calc_numbers)
    sub_1s_weekly = mkt_current.std_moves(1, 'weekly', calc_numbers)
    sub_15s_weekly = mkt_current.std_moves(1.5, 'weekly', calc_numbers)
    sub_2s_weekly = mkt_current.std_moves(2, 'weekly', calc_numbers)
    
    move = [1,-1, 1.5,-1.5, 2,-2]
    daily_moves = list(sub_1s_daily.Total) + list(sub_15s_daily.Total) + list(sub_2s_daily.Total)
    weekly_moves = list(sub_1s_weekly.Total) + list(sub_15s_weekly.Total) + list(sub_2s_weekly.Total)
    
    daily_prices = list(sub_1s_daily.Price) + list(sub_15s_daily.Price) + list(sub_2s_daily.Price)
    weekly_prices = list(sub_1s_weekly.Price) + list(sub_15s_weekly.Price) + list(sub_2s_weekly.Price)
    
    daily_frame = pd.DataFrame()
    daily_frame['Sigma'] = move
    daily_frame[mkt] = daily_moves
    daily_frame[mkt+'_Price'] = daily_prices
    
    weekly_frame = pd.DataFrame()
    weekly_frame['Sigma'] = move
    weekly_frame[mkt] = weekly_moves
    weekly_frame[mkt+'_Price'] = weekly_prices
    
    daily_frame = daily_frame.sort_values(by='Sigma').reset_index(drop=True)
    weekly_frame = weekly_frame.sort_values(by='Sigma').reset_index(drop=True)
    return daily_frame, weekly_frame

In [30]:
accu_daily, accu_weekly = sigma_moves('ACCU')
nzu_daily, nzu_weekly = sigma_moves('NZU')
eua_daily, eua_weekly = sigma_moves('EUA')
uka_daily, uka_weekly = sigma_moves('UKA')
cca_daily, cca_weekly = sigma_moves('CCA')

In [31]:
def sigma_moves_graph(accu,nzu,eua,uka,cca):
    sub = pd.DataFrame()
    sub['Sigma'] = accu.Sigma
    sub['ACCU'] = accu.ACCU
    sub['NZU'] = nzu.NZU
    sub['EUA'] = eua.EUA
    sub['UKA'] = uka.UKA
    sub['CCA'] = cca.CCA    
    
    prices = pd.DataFrame()
    prices['Sigma'] = accu.Sigma
    prices['ACCU'] = accu.ACCU_Price
    prices['NZU'] = nzu.NZU_Price
    prices['EUA'] = eua.EUA_Price
    prices['UKA'] = uka.UKA_Price
    prices['CCA'] = cca.CCA_Price    
    return sub, prices

fig_daily, prices_daily = sigma_moves_graph(accu_daily, nzu_daily, eua_daily, uka_daily, cca_daily)
fig_weekly, prices_weekly = sigma_moves_graph(accu_weekly, nzu_weekly, eua_weekly, uka_weekly, cca_daily)

In [32]:
fig = ff.create_table(prices_daily,height_constant=50)

for i in list(fig_daily)[1:]:
    fig.add_trace(go.Scatter(x=fig_daily.Sigma, y=fig_daily[i], name=i, xaxis='x2', yaxis='y2'))
    fig.update_xaxes(tickvals=[-2,-1.5,-1,1,1.5,2])
    
fig.update_layout(
    title_text = 'Daily Price Moves (check vols for EUA pnl calc)',
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.],'title':'Sigma'},
    yaxis2 = {'anchor': 'x2', 'title': 'PnL (AUD)'}
)

fig.show()

In [33]:
fig = ff.create_table(prices_weekly,height_constant=50)

for i in list(fig_weekly)[1:]:
    fig.add_trace(go.Scatter(x=fig_weekly.Sigma, y=fig_weekly[i], name=i, xaxis='x2', yaxis='y2'))
    fig.update_xaxes(tickvals=[-2,-1.5,-1,1,1.5,2])
    
fig.update_layout(
    title_text = 'Weekly Price Moves (check vols for eua pnl calc)',
    margin = {'t':50, 'b':100},
    xaxis = {'domain': [0, .5]},
    xaxis2 = {'domain': [0.6, 1.],'title':'Sigma'},
    yaxis2 = {'anchor': 'x2', 'title': 'PnL (AUD)'}
)

fig.show()


## Make the same graphs as above but for total portfolio allocation

# Position Summary Report
All PnL figures are in AUD  
Maybe put all the x-axes the same to better visualise scale

In [34]:
def generate_allocation_plots(mkt, mkt_name):
    sub = mkt[current_date][4]
    
    fig = go.Figure()
    for i in list(sub)[1:-1]:
        fig.add_trace(go.Bar(x=sub.Price, y=sub[i], name=i, base=0))
    fig.add_trace(go.Scatter(x=sub.Price, y=sub.Total_Allocation, mode='lines', name=mkt_name))
    fig.update_layout(barmode='stack')
    
    ## Add a horizontal line at y=30
    #fig.add_vline(x=40)
    return fig

In [35]:
def generate_summary_plots(mkt_data, data_type):
    if data_type==0:
        colname='Total_Pnl'
    elif data_type==1:
        colname='Total_Delta'
    elif data_type==2:
        colname='Option_Theta'
    elif data_type==3:
        colname='Option_Vega'
    fig = go.Figure()
    for i in list(range(0, len(dates_names))):
        date_value = dates_values[i]
        date_name = dates_names[i]
        fig.add_trace(go.Scatter(x=mkt_data[date_value][data_type].Price, y=mkt_data[date_value][data_type][colname], name=date_name))
    return fig

In [36]:
eua_d = generate_summary_plots(euas, 1)
eua_p = generate_summary_plots(euas, 0)
eua_t = generate_summary_plots(euas,2)
eua_v = generate_summary_plots(euas,3)

accu_d = generate_summary_plots(accus, 1)
accu_p = generate_summary_plots(accus, 0)
accu_t = generate_summary_plots(accus,2)
accu_v = generate_summary_plots(accus,3)

nzu_d = generate_summary_plots(nzus, 1)
nzu_p = generate_summary_plots(nzus, 0)
nzu_t = generate_summary_plots(nzus,2)
nzu_v = generate_summary_plots(nzus,3)

uka_d = generate_summary_plots(ukas, 1)
uka_p = generate_summary_plots(ukas, 0)
uka_t = generate_summary_plots(ukas,2)
uka_v = generate_summary_plots(ukas,3)

cca_d = generate_summary_plots(ccas, 1)
cca_p = generate_summary_plots(ccas, 0)
cca_t = generate_summary_plots(ccas,2)
cca_v = generate_summary_plots(ccas,3)

accu_a = generate_allocation_plots(accus, 'ACCU Allocation')
eua_a = generate_allocation_plots(euas, 'EUA Allocation')
nzu_a = generate_allocation_plots(nzus, 'NZU Allocation')
uka_a = generate_allocation_plots(nzus, 'UKA Allocation')
cca_a = generate_allocation_plots(nzus, 'CCA Allocation')

In [37]:
fig = make_subplots(rows=2, cols=3)
for i in range(0, len(accu_a.data)):
    fig.add_trace(accu_a.data[i], row=1, col=1)
    fig.add_trace(nzu_a.data[i], row=1, col=2)
    fig.add_trace(eua_a.data[i], row=1, col=3)
    fig.add_trace(uka_a.data[i], row=2, col=1)
    fig.add_trace(cca_a.data[i], row=2, col=2)    
fig.update_layout(title={'text': 'Net AUD Allocation (ACCU, NZU, EUA, UKA, CCA)','x': 0.5,'xanchor': 'center'})    
fig.show()

In [38]:
fig = make_subplots(rows=2, cols=3)
for i in range(0,len(eua_p.data)):
    fig.add_trace(accu_p.data[i], row=1, col=1)
    fig.add_trace(nzu_p.data[i], row=1, col=2)
    fig.add_trace(eua_p.data[i], row=1, col=3)
    fig.add_trace(uka_p.data[i], row=2, col=1)
    fig.add_trace(cca_p.data[i], row=2, col=2)    
    
fig.update_layout(title={'text': 'PnL (ACCU, NZU, EUA, UKA, CCA)','x': 0.5,'xanchor': 'center'})

fig.update_xaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="PnL", row=1, col=1)
fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="PnL", row=1, col=2)
fig.update_xaxes(title_text="Price", row=1, col=3)
#fig.update_yaxes(title_text="PnL", row=1, col=3)
fig.show()    

In [39]:
fig = make_subplots(rows=1, cols=3)
for i in range(0,len(eua_d.data)):
    fig.add_trace(accu_d.data[i], row=1, col=1)
    fig.add_trace(nzu_d.data[i], row=1, col=2)
    fig.add_trace(eua_d.data[i], row=1, col=3)
    
fig.update_layout(title={'text': 'Delta (ACCU, NZU, EUA)','x': 0.5,'xanchor': 'center'})    

fig.update_xaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Delta", row=1, col=1)
fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="PnL", row=1, col=2)
fig.update_xaxes(title_text="Price", row=1, col=3)
#fig.update_yaxes(title_text="PnL", row=1, col=3)
fig.show()    

## TO DO:
Need to aggregate the theta  
Need to increase the figsize

In [40]:
fig = make_subplots(rows=3, cols=2)
for i in range(0,len(eua_t.data)):
    fig.add_trace(accu_t.data[i], row=1, col=1)
    fig.add_trace(nzu_t.data[i], row=1, col=2)
    fig.add_trace(eua_t.data[i], row=2, col=1)
    fig.add_trace(uka_t.data[i], row=2, col=2)
    fig.add_trace(cca_t.data[i], row=3, col=1)
    
fig.update_layout(title={'text': 'Theta (ACCU, NZU, EUA, UKA, CCA)','x': 0.5,'xanchor': 'center'})    

fig.update_xaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Theta", row=1, col=1)
fig.update_xaxes(title_text="Price", row=1, col=2)
#fig.update_yaxes(title_text="PnL", row=1, col=2)
fig.update_xaxes(title_text="Price", row=1, col=3)
#fig.update_yaxes(title_text="PnL", row=1, col=3)
fig.show()   